# Reyes-tools: Parameter Tuning for Electron Diffraction Quality Determination

This notebook helps you configure and optimize parameters for autonomous MicroED diffraction quality assessment. Use this tool to tune microscope-specific parameters after REyes is installed and run for the first time.

**IMPORTANT**: This implementation matches the actual REyes package processing algorithm with critical uint16 overflow fixes applied.

## How the Analysis Works

The `DiffractionAnalyzer` uses a two-stage approach to evaluate diffraction quality:

### Stage 1: Diffraction Peak Detection
- **Gaussian Filtering**: Applies light and harsh Gaussian blurs to enhance peaks and remove background
- **uint16 Overflow Protection**: Converts uint16 data to float64 BEFORE processing to prevent overflow
- **Adaptive Thresholding**: Uses statistical thresholds to identify significant Diffraction Peaks
- **Region Analysis**: Counts and localizes detected peaks while excluding the central beam
- **Edge Masking**: Ignores 3% border regions to avoid artifacts (matches REyes implementation)

### Stage 2: LQP (Lattice Quality Peaks) Assessment  
- **Fourier Transform**: Analyzes the spatial frequency content of detected peaks
- **LQP Detection**: Evaluates how regular/crystalline the diffraction pattern is using hardcoded FT parameters
- **DQI Calculation**: Computes Diffraction Quality Index (DQI = LQP / Diffraction Peaks)
- **Quality Classification**: Uses DQI threshold comparison for final classification

### Quality Categories
- **Good diffraction**: Many peaks (10+) with good lattice regularity (DQI >= good_rule)
- **Bad diffraction**: Many peaks (10+) but poor lattice quality (DQI < good_rule)
- **Poor diffraction**: Few peaks (3-9) but detectable pattern
- **No diffraction**: Very few peaks (< 3) detected
- **Grid**: Low-intensity regions (empty grid squares or carbon film)

## Key Features

- **Parameter Tuning**: Configure analysis for different microscope setups
- **REyes Compatibility**: Algorithms match the actual REyes implementation
- **uint16 Overflow Protection**: Critical fix for accurate peak detection on uint16 MRC files
- **Batch Processing**: Analyze entire directories automatically
- **Comprehensive Visualization**: 9-panel analysis plots for each frame
- **Statistical Reporting**: Detailed summaries and success rates
- **Organized Output**: Results saved in structured directory hierarchy
- **Correct Nomenclature**: Uses LQP (Lattice Quality Peaks) and DQI terminology

## Installation Recomendations

If you run this notebook on a computer different from where you ran REyes, ensure you have the required packages installed:

```bash
# Create a new environment (recommended)
micromamba create -n reyes_env python=3.11
micromamba activate reyes_env

# Install required packages
pip install hyperspy numpy matplotlib scipy scikit-image ipykernel

# Note: ipykernel is lighter than full jupyter package
# If you need full Jupyter interface, replace ipykernel with jupyter
```

## Directory Structure Expected

The notebook expects your data to be organized as follows:
```
your_project_directory/
├── diffraction_quality.ipynb  (this notebook)
├── folder_1/                   (containing .mrc files)
│   ├── image1.mrc
│   ├── image2.mrc
│   └── ...
├── folder_2/                   (containing .mrc files)
│   ├── image1.mrc
│   └── ...
└── snapshots/                 (special folder name)
    ├── image1.mrc
    └── ...
```

## Parameter Configuration Guide

### Primary Analysis Parameters (Tunable)

**`light_sigma_prim`** (default: 1)
- Controls light Gaussian blur for spot enhancement
- **Typical range**: 0.5-3
- **Troubleshooting**: Increase if spots appear too sharp/noisy

**`harsh_sigma_prim`** (default: 5) 
- Controls harsh Gaussian blur for background subtraction
- **Typical range**: 3-7
- **Troubleshooting**: Increase if background subtraction is poor

**`threshold_std_prim`** (default: 5) **MOST CRITICAL**
- Standard deviation multiplier for spot detection threshold
- **This is the parameter you'll most likely need to tune**
- **Typical range**: 3-8 for most data
- **If too sensitive (noise spots)**: Increase to 7-15
- **If missing real peaks**: Decrease to 3-4

**`min_pixels_prim`** (default: 5)
- Minimum area (in pixels) for detected spots
- **Typical range**: 3-10
- **High magnification**: May need to reduce to 3-4
- **Low magnification**: May need to increase to 6-10

### Quality Assessment Parameters

**`good_rule`** (default: 3)
- DQI threshold for good vs bad diffraction classification
- **Current value provides balanced classification for most setups**
- **More lenient**: Reduce to 2 (more "Good" classifications)
- **More strict**: Increase to 4-10 (fewer "Good" classifications)

### Fixed Parameters (Don't Change)

**`exclude_center`** (Fixed: 25%)
- Excludes central 25% to avoid counting central beam
- Fixed in REyes implementation

**FT Processing Parameters** (Fixed in REyes):
- `harsh_sigma_ft`: 20
- `threshold_std_ft`: 3  
- `min_pixels_ft`: 1

## Tuning Process

1. **Start with defaults** (threshold_std_prim: 5, good_rule: 3)
2. **Process a small test dataset** (20-50 different images)
3. **Check visualizations** in `output/` directory
4. **Adjust values as needed**
5. **Re-run and validate** on larger dataset

## Results Interpretation

### Understanding the Visualization Output

When you run the analysis, each frame generates a 9-panel visualization saved in `output/[folder_name]/`. Here's how to interpret the results:

**Row 1 - Gaussian Filtering Steps:**
- **Panel 1 - Original Pattern**: Raw detector image
- **Panel 2 - Light Blur**: After light Gaussian blur for spot enhancement
- **Panel 3 - Harsh Blur**: After harsh Gaussian blur for background subtraction

**Row 2 - Diffraction Peak Detection:**
- **Panel 4 - Difference Image**: Light - Harsh blur (should NOT show overflow artifacts!)
- **Panel 5 - Detection Mask**: Binary image of detected regions
- **Panel 6 - Detected Peaks**: Red circles overlaid on original

**Row 3 - LQP Analysis:**
- **Panel 7 - FT of Detection Mask**: Fourier transform for LQP analysis
- **Panel 8 - LQP Detection Mask**: Binary image from FT analysis
- **Panel 9 - LQP Results**: Blue circles showing detected LQP with DQI value

### Critical Fix Verification

**Panel 4 (Difference Image) is KEY for verifying the uint16 fix:**
- **GOOD**: Smooth grayscale image with reasonable contrast
- **BAD**: White/bright patches or entire image is black indicate overflow (max values ~65535)
- **If you see overflow**: Check pixel values for images

**Panel 6 (Detected Peaks) Quality Check:**
- **Good result**: Red circles on actual Diffraction Peaks only
- **Too sensitive**: Red circles on background noise → Increase `threshold_std_prim`
- **Too insensitive**: Missing obvious peaks → Decrease `threshold_std_prim`

### Quality Classifications

**Good diffraction** (Target for data collection):
- 10+ well-defined Diffraction Peaks
- Regular crystalline pattern (high DQI >= good_rule)

**Bad diffraction** (Many peaks but poor lattice quality):
- 10+ Diffraction Peaks but low DQI < good_rule
- Often indicates polycrystalline or twinned samples
- Sometimes good patterns might be misclassified here

**Poor diffraction** (Usable but limited):
- 3-9 peaks detected
- May indicate small crystals or partial order

**No diffraction** (Empty or amorphous):
- <3 peaks detected
- Empty grid squares or non-crystalline material

**Grid** (Empty areas):
- Low overall intensity
- Empty grid squares, torn areas, or thick carbon support

### Troubleshooting

**Issue: All patterns classified as "Bad diffraction"**
- **Possible cause**: `good_rule` too strict for your crystals
- **Check**: Visually verify diffraction quality before decreasing `good_rule` from 3 to 2

**Issue: Too many "Good diffraction" classifications**
- **Likely cause**: `good_rule` too lenient
- **Solution**: Increase `good_rule` from 3 to 5 or higher

**Issue: Inconsistent results between similar images**
- **Likely cause**: threshold_std_prim on the border of sensitivity
- **Solution**: Adjust threshold_std_prim by ±1 and test consistency

**Issue: Seeing overflow artifacts in Panel 4**
- **Critical problem**: uint16 overflow fix not working
- **Check**: Data type conversion in parse_mrc method
- **Solution**: Ensure data.astype(np.float64) is applied before processing

**Issue: Visually good diffraction patterns classified as "Bad"**
- **Possible causes**: Parameter tuning needed or unexpected pixel value behavior
- **Action required**: Check the log files for pixel value statistics and intensity distributions
- **If inconsistencies observed**: Report details (pixel value ranges, intensity statistics, specific pattern examples) to the development team for investigation

In [ ]:
import os
import re
import logging
import csv
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass
from datetime import datetime

import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.measure import label, regionprops_table


@dataclass
class DiffractionResult:
    """Container for diffraction analysis results with updated nomenclature."""
    n_diffraction_spots: int  # Number of detected Diffraction Peaks (matches dif_map.py)
    n_pattern_spots: int     # Number of pattern spots (matches dif_map.py nomenclature)
    total_sum: float         # Total image intensity
    dqi: float              # Diffraction Quality Index (LQP / Diffraction Peaks)
    quality: str            # Quality classification


class DiffractionAnalyzer:
    """
    Automated electron diffraction pattern analysis for MicroED experiments.
    
    This class provides comprehensive analysis of electron diffraction patterns including:
    - Automated peak detection using Gaussian filtering with uint16 overflow protection
    - Quality classification (Good, Poor, Bad, No diffraction, Grid)
    - LQP assessment via Fourier transform analysis
    - DQI (Diffraction Quality Index) calculation
    - Batch processing of MRC files
    - Visualization and statistical reporting
    - Pixel value statistics logging and CSV export for debugging and optimization
    
    IMPORTANT: This implementation matches the actual REyes package dif_map.py processing
    with critical uint16 overflow fixes applied.
    """
    
    def __init__(self, params: Optional[Dict[str, Any]] = None):
        """
        Initialize the DiffractionAnalyzer with analysis parameters.
        
        Args:
            params: Dictionary of analysis parameters. If None, uses default 
                   values optimized for typical MicroED data.
        """
        # Default parameters - good starting point for most microscope setups
        self.params = {
            # Primary analysis parameters
            'light_sigma_prim': 1,        # Light Gaussian blur for peak enhancement
            'harsh_sigma_prim': 5,        # Harsh Gaussian blur for background subtraction
            'threshold_std_prim': 5,      # Std dev multiplier for peak detection threshold
            'min_pixels_prim': 5,         # Minimum area for peak detection
            'exclude_center': 25,         # Percentage of center to exclude from peak counting
            
            # Quality assessment parameters
            'good_rule': 3,               # DQI threshold for good vs bad diffraction
            'grid_rule': 3,               # Threshold for grid/empty region identification
            
            # FT processing parameters (hardcoded in REyes)
            'harsh_sigma_ft': 20,         # Gaussian blur for FT analysis
            'threshold_std_ft': 3,        # Threshold for pattern detection in FT
            'min_pixels_ft': 1,           # Minimum size for FT patterns
        }
        
        # Update with provided parameters if any
        if params:
            self.params.update(params)
        
        # Initialize results storage
        self.diffraction_quality: Dict[str, DiffractionResult] = {}
        
        # Initialize pixel statistics storage
        self.pixel_statistics: Dict[str, Dict[str, Any]] = {}
        
        # Create output directory
        self.output_dir = Path('output')
        self.output_dir.mkdir(exist_ok=True)
        
        # Setup custom logging for diffraction quality analysis
        self.stats_log_path = self.output_dir / 'DQ_optimizer.log'
        self.logger = self._setup_logger()
        
        # Counter for frame numbering in output files
        self.frame_counter = 0
        
        # Track if conversion message has been logged
        self._conversion_logged = False

    def _setup_logger(self):
        """Setup custom logger with header information only once."""
        # Create custom logger
        logger = logging.getLogger('diffraction_analyzer')
        logger.setLevel(logging.INFO)
        
        # Clear existing handlers
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)
        
        # Create file handler with custom formatter
        file_handler = logging.FileHandler(self.stats_log_path, mode='w')
        
        # Custom formatter without timestamp for individual entries
        formatter = logging.Formatter('%(message)s')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
        
        # Write header information once
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        logger.info("DIFFRACTION QUALITY OPTIMIZER LOG")
        logger.info("=" * 60)
        logger.info(f"Analysis started: {current_time}")
        logger.info("")
        logger.info("PROCESSING PARAMETERS:")
        for key, value in self.params.items():
            logger.info(f"  {key}: {value}")
        logger.info("")
        logger.info("DIFFRACTION QUALITY ANALYSIS RESULTS:")
        logger.info("-" * 60)
        
        return logger

    def log_pixel_statistics_csv(self, data: np.ndarray, light_blur_data: np.ndarray, 
                                harsh_blur_data: np.ndarray, difference_data: np.ndarray, 
                                folder_name: Optional[str], file_name: str):
        """
        Log pixel statistics for all image processing steps to a CSV file.
        This matches the functionality from the _fixed version.
        
        Args:
            data: Original image data
            light_blur_data: Light Gaussian blur result
            harsh_blur_data: Harsh Gaussian blur result  
            difference_data: Difference image (light - harsh)
            folder_name: Name of containing folder
            file_name: Name of MRC file
        """
        # Create log directory if it doesn't exist
        log_dir = self.output_dir / folder_name if folder_name else self.output_dir
        log_dir.mkdir(exist_ok=True)
        csv_file = log_dir / 'pixel_statistics_detailed.csv'
        
        # Calculate statistics for each image
        images = {
            'original_pattern': data,
            'light_blur': light_blur_data,
            'harsh_blur': harsh_blur_data,
            'difference_image': difference_data
        }
        
        # Check if file exists to determine if we need header
        file_exists = csv_file.exists()
        
        with open(csv_file, 'a', newline='') as csvfile:
            fieldnames = ['file_name', 'image_type', 'min_pixel', 'max_pixel', 'mean_pixel', 'std_pixel']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            if not file_exists:
                writer.writeheader()
            
            # Write statistics for each image type
            for img_name, img_data in images.items():
                writer.writerow({
                    'file_name': file_name,
                    'image_type': img_name,
                    'min_pixel': float(np.min(img_data)),
                    'max_pixel': float(np.max(img_data)),
                    'mean_pixel': float(np.mean(img_data)),
                    'std_pixel': float(np.std(img_data))
                })

    def log_pixel_statistics(self, file_name: str, data: np.ndarray, 
                           light_blur_data: np.ndarray, harsh_blur_data: np.ndarray, 
                           difference_data: np.ndarray, threshold_value: float,
                           n_diffraction_spots: int = None, n_pattern_spots: int = None, 
                           dqi: float = None, quality: str = None, total_sum: float = None):
        """
        Log comprehensive pixel value statistics and analysis results for debugging and optimization.
        """
        stats = {
            'file_name': file_name,
            'original_dtype': str(data.dtype),
            'original_range': [float(np.min(data)), float(np.max(data))],
            'original_mean': float(np.mean(data)),
            'original_std': float(np.std(data)),
            'light_blur_range': [float(np.min(light_blur_data)), float(np.max(light_blur_data))],
            'harsh_blur_range': [float(np.min(harsh_blur_data)), float(np.max(harsh_blur_data))],
            'difference_range': [float(np.min(difference_data)), float(np.max(difference_data))],
            'difference_mean': float(np.mean(difference_data)),
            'difference_std': float(np.std(difference_data)),
            'threshold_value': float(threshold_value),
            'pixels_above_threshold': int(np.sum(difference_data > threshold_value)),
            'total_pixels': int(data.size)
        }
        
        # Store statistics
        self.pixel_statistics[file_name] = stats
        
        # Log to file (no timestamps, clean format)
        self.logger.info(f"ANALYSIS RESULTS for {file_name}")
        
        # Log analysis results if provided
        if total_sum is not None:
            self.logger.info(f"  Total intensity: {total_sum:.0f}")
        if n_diffraction_spots is not None:
            self.logger.info(f"  Diffraction Peaks: {n_diffraction_spots}")
        if n_pattern_spots is not None:
            self.logger.info(f"  LQP: {n_pattern_spots}")
        if dqi is not None:
            self.logger.info(f"  DQI: {dqi:.3f}")
        if quality is not None:
            self.logger.info(f"  Classification: {quality}")
        
        # Log pixel statistics
        self.logger.info(f"  Original data: dtype={stats['original_dtype']}, "
                        f"range=[{stats['original_range'][0]:.1f}, {stats['original_range'][1]:.1f}], "
                        f"mean={stats['original_mean']:.1f}, std={stats['original_std']:.1f}")
        self.logger.info(f"  Light blur range: [{stats['light_blur_range'][0]:.1f}, {stats['light_blur_range'][1]:.1f}]")
        self.logger.info(f"  Harsh blur range: [{stats['harsh_blur_range'][0]:.1f}, {stats['harsh_blur_range'][1]:.1f}]")
        self.logger.info(f"  Difference image: range=[{stats['difference_range'][0]:.1f}, {stats['difference_range'][1]:.1f}], "
                        f"mean={stats['difference_mean']:.2f}, std={stats['difference_std']:.2f}")
        self.logger.info(f"  Threshold: {stats['threshold_value']:.2f}")
        self.logger.info(f"  Pixels above threshold: {stats['pixels_above_threshold']}/{stats['total_pixels']} "
                        f"({100*stats['pixels_above_threshold']/stats['total_pixels']:.1f}%)")
        
        # Check for potential overflow issues
        if stats['original_dtype'] in ['uint16', 'uint8', 'uint32']:
            if stats['original_dtype'] == 'uint16' and stats['difference_range'][1] > 60000:
                self.logger.info(f"  WARNING: HIGH DIFFERENCE VALUES detected! Max difference: {stats['difference_range'][1]:.1f}")
                self.logger.info(f"  WARNING: This suggests uint16 overflow may have occurred")
            elif stats['original_dtype'] == 'uint8' and stats['difference_range'][1] > 240:
                self.logger.info(f"  WARNING: HIGH DIFFERENCE VALUES detected! Max difference: {stats['difference_range'][1]:.1f}")
        
        self.logger.info("  " + "-" * 60)

    def parse_mrc(self, mrc_file: str) -> np.ndarray:
        """
        Load MRC file using HyperSpy with uint16 overflow protection.
        Matches dif_map.py parse_mrc method exactly.
        
        Args:
            mrc_file: Path to the MRC file
            
        Returns:
            Image data array as float64 to prevent overflow
        """
        signal = hs.load(mrc_file)
        data = signal.data
        
        # CRITICAL FIX: Convert unsigned integer data to float64 early to prevent overflow
        if data.dtype in [np.uint8, np.uint16, np.uint32]:
            if not self._conversion_logged:
                print(f"Converting {data.dtype} to float64 to prevent overflow")
                self._conversion_logged = True
            data = data.astype(np.float64)
        
        return data

    def create_binary_image(self, data: np.ndarray, light_sigma: float, 
                          harsh_sigma: float, threshold_std: float, 
                          ft_done: bool = False) -> np.ndarray:
        """
        Create binary image for peak detection using Gaussian filtering and thresholding.
        Matches REyes ImageProcessor.create_binary_image() method with overflow protection.
        """
        # CRITICAL FIX: Convert uint16 to float64 to prevent overflow
        if data.dtype in [np.uint8, np.uint16, np.uint32]:
            data = data.astype(np.float64)
        
        if ft_done:
            # For FT data, use hardcoded values as in REyes
            harsh_blur_data = gaussian_filter(data, sigma=20)
            difference_data = data - harsh_blur_data
            threshold_value = np.mean(difference_data) + (3 * np.std(difference_data))
        else:
            # Standard processing: subtract harsh blur from light blur
            light_blur_data = gaussian_filter(data, sigma=light_sigma)
            harsh_blur_data = gaussian_filter(data, sigma=harsh_sigma)
            difference_data = light_blur_data - harsh_blur_data
            threshold_value = np.mean(difference_data) + (threshold_std * np.std(difference_data))

        binary_image = difference_data > threshold_value
        
        # Mask 3% on all four sides (matches REyes)
        mask_size_x = int(binary_image.shape[1] * 0.03)
        mask_size_y = int(binary_image.shape[0] * 0.03)
        
        binary_image[:, :mask_size_x] = False
        binary_image[:, -mask_size_x:] = False
        binary_image[:mask_size_y, :] = False
        binary_image[-mask_size_y:, :] = False
        
        return binary_image

    def get_binary_ft(self, data: np.ndarray) -> np.ndarray:
        """
        Generate Fourier transform magnitude (matches REyes implementation).
        """
        ft_data = np.fft.fftshift(np.fft.fft2(data))
        ft_magnitude = np.log(np.abs(ft_data) + 1)
        return ft_magnitude

    def get_centroids(self, binary_image: np.ndarray, min_pixels: int, 
                     exclude_center: Optional[float] = None, 
                     ft_done: bool = False) -> Tuple[List[Tuple[float, float]], int]:
        """
        Extract centroids of detected regions from binary image.
        Matches REyes ImageProcessor.get_centroids() method exactly.
        """
        labeled_array = label(binary_image)
        properties = regionprops_table(labeled_array, properties=('centroid', 'area'))
        
        # Use min_pixels=1 for FT patterns, config value otherwise (matches dif_map.py)
        min_pixels_to_use = 1 if ft_done else min_pixels
        
        # Filter by minimum area
        centroids = [(x, y) for x, y, area in zip(properties['centroid-0'], 
                                                 properties['centroid-1'],
                                                 properties['area']) 
                    if area >= min_pixels_to_use]
        
        # For diffraction analysis, exclude central region (avoid central beam)
        if not ft_done and exclude_center is not None:
            image_shape = binary_image.shape
            center_x, center_y = image_shape[1] // 2, image_shape[0] // 2
            min_distance = min(center_x, center_y)
            exclude_radius = (exclude_center / 100) * min_distance
            
            filtered_centroids = [
                centroid for centroid in centroids
                if not (center_x - exclude_radius < centroid[1] < center_x + exclude_radius and
                        center_y - exclude_radius < centroid[0] < center_y + exclude_radius)
            ]
        else:
            filtered_centroids = centroids
        
        return filtered_centroids, len(filtered_centroids)

    def _classify_diffraction(self, n_dif_spots: int, n_pat_spots: int) -> str:
        """
        Classify diffraction quality based on peak counts and DQI.
        Matches REyes DiffractionAnalyzer._classify_diffraction() method exactly.
        """
        if n_dif_spots < 3:
            return 'No diffraction'
        elif n_dif_spots < 10:
            return 'Poor diffraction'
        elif self.params['good_rule'] * n_dif_spots > n_pat_spots:
            # DQI (n_pat_spots/n_dif_spots) is below good_rule threshold
            return 'Bad diffraction'
        else:
            # DQI (n_pat_spots/n_dif_spots) meets good_rule threshold
            return 'Good diffraction'

    def process_file(self, file_path: str, visualize: bool = True, 
                    folder_name: Optional[str] = None) -> Optional[DiffractionResult]:
        """
        Process a single MRC file for diffraction quality analysis.
        Matches dif_map.py process_mrc_file method logic exactly.
        """
        file_name = os.path.basename(file_path)
        print(f"Processing: {file_name}")

        try:
            # Load and analyze image data with overflow protection
            data = self.parse_mrc(file_path)
            total_sum = np.sum(data)

            # Generate Gaussian blurred images for analysis
            light_blur_data = gaussian_filter(data, sigma=self.params['light_sigma_prim'])
            harsh_blur_data = gaussian_filter(data, sigma=self.params['harsh_sigma_prim'])
            difference_data = light_blur_data - harsh_blur_data
            
            # Calculate threshold for pixel statistics logging
            threshold_value = np.mean(difference_data) + (self.params['threshold_std_prim'] * np.std(difference_data))
            
            # Log pixel statistics to CSV (matches _fixed version functionality)
            self.log_pixel_statistics_csv(data, light_blur_data, harsh_blur_data, 
                                         difference_data, folder_name, file_name)

            # Create primary binary image and detect Diffraction Peaks
            primary_binary_image = self.create_binary_image(
                data, 
                self.params['light_sigma_prim'],
                self.params['harsh_sigma_prim'],
                self.params['threshold_std_prim'],
                ft_done=False
            )

            primary_centroids, n_dif_spots = self.get_centroids(
                primary_binary_image,
                self.params['min_pixels_prim'],
                exclude_center=25,
                ft_done=False
            )

            # Process FT pattern for LQP detection (matches dif_map.py exactly)
            ft_binary_image = self.get_binary_ft(primary_binary_image)
            secondary_binary_image = self.create_binary_image(
                ft_binary_image,
                None,  # Not used for FT processing
                self.params['harsh_sigma_ft'],
                self.params['threshold_std_ft'],
                ft_done=True
            )

            secondary_centroids, n_pat_spots = self.get_centroids(
                secondary_binary_image,
                self.params['min_pixels_ft'],
                exclude_center=None,
                ft_done=True
            )

            # Calculate Diffraction Quality Index (DQI)
            dqi = n_pat_spots / n_dif_spots if n_dif_spots > 0 else 0

            # Classify diffraction quality using DQI (matches dif_map.py exactly)
            quality = self._classify_diffraction(n_dif_spots, n_pat_spots)

            # Log pixel statistics and analysis results for debugging and optimization
            self.log_pixel_statistics(file_name, data, light_blur_data, harsh_blur_data, 
                                    difference_data, threshold_value, n_dif_spots, n_pat_spots, 
                                    dqi, quality, total_sum)

            # Create result object (matches dif_map.py DiffractionResult)
            result = DiffractionResult(
                n_diffraction_spots=n_dif_spots,
                n_pattern_spots=n_pat_spots,
                total_sum=total_sum,
                dqi=dqi,
                quality=quality
            )

            # Store results
            self.diffraction_quality[file_name] = result

            # Generate visualization if requested
            if visualize:
                self.visualize_results(
                    data, 
                    primary_binary_image, 
                    ft_binary_image,
                    secondary_binary_image,
                    primary_centroids, 
                    secondary_centroids,
                    folder_name,
                    file_name,
                    light_blur_data,
                    harsh_blur_data,
                    difference_data
                )

            return result

        except Exception as e:
            print(f"Error processing {file_name}: {str(e)}")
            return None

    def visualize_results(self, data, primary_binary, ft_image, secondary_binary, 
                         primary_centroids=None, secondary_centroids=None, 
                         folder_name=None, file_name=None,
                         light_blur_data=None, harsh_blur_data=None, 
                         difference_data=None):
        """
        Create comprehensive 3x3 visualization of analysis results.
        """
        fig = plt.figure(figsize=(15, 15))
        
        # Generate informative title
        n_dif = len(primary_centroids) if primary_centroids else 0
        n_pat = len(secondary_centroids) if secondary_centroids else 0
        dqi = n_pat / n_dif if n_dif > 0 else 0
        quality = self._classify_diffraction(n_dif, n_pat)
        
        title_line1 = f'Diffraction Peaks: {n_dif} | LQP: {n_pat} | DQI: {dqi:.3f} | Assessment: {quality}'
        title_line2 = f'File: {file_name}' if file_name else ''
        
        plt.suptitle(
            f'{title_line1}\n{title_line2}',
            fontsize=14, y=0.95
        )
        
        # Create 3x3 subplot grid
        gs = plt.GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.2)
        
        # Generate Gaussian smoothed images if not provided
        if light_blur_data is None:
            light_blur_data = gaussian_filter(data, sigma=self.params['light_sigma_prim'])
        if harsh_blur_data is None:
            harsh_blur_data = gaussian_filter(data, sigma=self.params['harsh_sigma_prim'])
        if difference_data is None:
            difference_data = light_blur_data - harsh_blur_data
        
        # Set consistent intensity scaling
        vmin, vmax = np.percentile(data, 1), np.percentile(data, 99)
        
        # Row 1: Gaussian Filtering Steps
        
        # 1. Original diffraction pattern
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.imshow(data, cmap='gray', vmin=vmin, vmax=vmax)
        ax1.set_title('Original Pattern', fontsize=12)
        ax1.axis('off')
        
        # 2. After light sigma Gaussian blur
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.imshow(light_blur_data, cmap='gray', vmin=vmin, vmax=vmax)
        ax2.set_title(f'Light Blur (sigma={self.params["light_sigma_prim"]})', fontsize=12)
        ax2.axis('off')
        
        # 3. After harsh sigma Gaussian blur
        ax3 = fig.add_subplot(gs[0, 2])
        ax3.imshow(harsh_blur_data, cmap='gray', vmin=vmin, vmax=vmax)
        ax3.set_title(f'Harsh Blur (sigma={self.params["harsh_sigma_prim"]})', fontsize=12)
        ax3.axis('off')
        
        # Row 2: Diffraction Peak Detection Process
        
        # 4. Difference image (light - harsh)
        ax4 = fig.add_subplot(gs[1, 0])
        diff_vmin, diff_vmax = np.percentile(difference_data, 5), np.percentile(difference_data, 95)
        ax4.imshow(difference_data, cmap='gray', vmin=diff_vmin, vmax=diff_vmax)
        ax4.set_title('Difference Image\n(Light - Harsh)', fontsize=12)
        ax4.axis('off')
        
        # 5. Binary mask from diffraction peak detection
        ax5 = fig.add_subplot(gs[1, 1])
        ax5.imshow(primary_binary, cmap='gray')
        ax5.set_title(f'Diffraction Peak Detection\n(threshold={self.params["threshold_std_prim"]}sigma)', fontsize=12)
        ax5.axis('off')
        
        # 6. Detected Diffraction Peaks overlaid on original
        ax6 = fig.add_subplot(gs[1, 2])
        ax6.imshow(data, cmap='gray', vmin=vmin, vmax=vmax)
        if primary_centroids:
            ax6.scatter(
                [c[1] for c in primary_centroids], [c[0] for c in primary_centroids],
                color='red', s=20, marker='o', facecolors='none', edgecolors='red', linewidth=1.5
            )
        ax6.set_title(f'Detected Diffraction Peaks\n({n_dif} peaks)', fontsize=12)
        ax6.axis('off')
        
        # Row 3: LQP Analysis
        
        # 7. Fourier transform for LQP analysis
        ax7 = fig.add_subplot(gs[2, 0])
        ax7.imshow(ft_image, cmap='gray')
        ax7.set_title('FT of Diffraction Peak\nDetection Mask', fontsize=12)
        ax7.axis('off')
        
        # 8. Binary mask from FT analysis for LQP detection
        ax8 = fig.add_subplot(gs[2, 1])
        ax8.imshow(secondary_binary, cmap='gray')
        ax8.set_title('LQP Detection Mask', fontsize=12)
        ax8.axis('off')
        
        # 9. LQP with DQI
        ax9 = fig.add_subplot(gs[2, 2])
        ax9.imshow(secondary_binary, cmap='gray')
        if secondary_centroids:
            ax9.scatter(
                [c[1] for c in secondary_centroids], [c[0] for c in secondary_centroids],
                color='blue', s=20, marker='o', facecolors='none', edgecolors='blue', linewidth=1.5
            )
        ax9.set_title(f'LQP: {n_pat} | DQI: {dqi:.3f}', fontsize=12)
        ax9.axis('off')
        
        plt.tight_layout()
        
        # Save with organized directory structure
        if folder_name:
            save_dir = self.output_dir / folder_name
            save_dir.mkdir(exist_ok=True)
            save_path = save_dir / f'diffraction_analysis_frame_{self.frame_counter:03d}.png'
        else:
            save_path = self.output_dir / f'diffraction_analysis_frame_{self.frame_counter:03d}.png'
        
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        self.frame_counter += 1
        plt.close()

    def update_diffraction_quality(self):
        """
        Post-process results to identify grid/empty regions based on intensity.
        Matches dif_map.py update_diffraction_quality method exactly.
        """
        sums = [result.total_sum for result in self.diffraction_quality.values()]
        if not sums:
            return
            
        mean_sum = np.mean(sums)
        threshold = mean_sum / self.params['grid_rule']

        # Reclassify low-intensity regions as grid
        for file_name, result in self.diffraction_quality.items():
            if result.total_sum < threshold:
                updated_result = DiffractionResult(
                    n_diffraction_spots=0,
                    n_pattern_spots=0,
                    total_sum=result.total_sum,
                    dqi=0,
                    quality='Grid'
                )
                self.diffraction_quality[file_name] = updated_result

    def print_summary_statistics(self):
        """Print comprehensive summary statistics."""
        if not self.diffraction_quality:
            print("No results to summarize.")
            return

        # Count occurrences of each quality category
        quality_counts = {
            'No diffraction': 0,
            'Poor diffraction': 0,
            'Good diffraction': 0,
            'Bad diffraction': 0,
            'Grid': 0
        }

        for result in self.diffraction_quality.values():
            quality = result.quality
            if quality in quality_counts:
                quality_counts[quality] += 1

        total = sum(quality_counts.values())

        # Print summary
        print("\nDIFFRACTION ANALYSIS SUMMARY")
        print("=" * 40)
        
        for quality, count in quality_counts.items():
            percentage = (count / total * 100) if total > 0 else 0
            print(f"{quality:>18}: {count:>3} ({percentage:5.1f}%)")
        
        print(f"{'Total analyzed':>18}: {total:>3}")
        
        # Calculate useful metrics
        if total > 0:
            usable = quality_counts['Good diffraction'] + quality_counts['Poor diffraction']
            usable_pct = (usable / total * 100)
            print(f"\nUsable patterns: {usable}/{total} ({usable_pct:.1f}%)")
            
            # Print average DQI information
            avg_dqi = np.mean([r.dqi for r in self.diffraction_quality.values() if r.dqi > 0])
            if not np.isnan(avg_dqi):
                print(f"Average DQI: {avg_dqi:.3f}")

    def process_directory(self, directory_path: str, visualize: bool = True, 
                         max_files: Optional[int] = None):
        """
        Process all MRC files in a directory.
        """
        self.frame_counter = 0
        folder_name = os.path.basename(directory_path)
        
        directory_path = Path(directory_path)
        if not directory_path.exists():
            print(f"Directory not found: {directory_path}")
            return self.diffraction_quality

        mrc_files = list(directory_path.glob("*.mrc"))
        
        if not mrc_files:
            print(f"No MRC files found in {directory_path}")
            return self.diffraction_quality

        # Limit number of files if specified
        if max_files is not None and len(mrc_files) > max_files:
            mrc_files = sorted(mrc_files)[:max_files]
            print(f"Processing first {max_files} of {len(list(directory_path.glob('*.mrc')))} files")
        else:
            mrc_files = sorted(mrc_files)
            print(f"Processing {len(mrc_files)} files in {folder_name}")
        
        # Process each file
        processed_count = 0
        for mrc_file in mrc_files:
            result = self.process_file(str(mrc_file), visualize, folder_name)
            if result is not None:
                processed_count += 1

        # Post-process and generate summary (matches dif_map.py)
        self.update_diffraction_quality()
        print(f"Completed processing {processed_count} files")
        self.print_summary_statistics()
        
        print(f"\nFILES CREATED:")
        print(f"  * output/{folder_name}/pixel_statistics_detailed.csv - Detailed pixel stats for each processing step")
        print(f"  * output/DQ_optimizer.log - Complete diffraction quality analysis log")
        
        return self.diffraction_quality

print("DiffractionAnalyzer class loaded with pixel statistics logging and CSV export functionality")
print("This implementation matches dif_map.py logic exactly.")

In [ ]:
# Configure analysis parameters
# These default values are a good starting point for most microscope setups

params = {
    # PRIMARY ANALYSIS PARAMETERS (Most likely to need tuning)
    'light_sigma_prim': 1,        # Spot enhancement
    'harsh_sigma_prim': 5,        # Background subtraction
    'threshold_std_prim': 6,      # CRITICAL: Spot detection sensitivity
    'min_pixels_prim': 5,         # Minimum spot size
    
    # QUALITY ASSESSMENT PARAMETERS
    'good_rule': 3,               # DQI threshold for quality classification
    'grid_rule': 3,               # Grid detection
    
    # ADVANCED PARAMETERS (fixed - don't change)
    'exclude_center': 25,         # Central beam exclusion
    'harsh_sigma_ft': 20,         # FT processing
    'threshold_std_ft': 3,        # FT threshold
    'min_pixels_ft': 1,           # FT min pixels
}

print("PARAMETER CONFIGURATION")
print("=" * 30)

for key, value in params.items():
    if key in ['threshold_std_prim', 'good_rule']:
        print(f"{key:>20}: {value:>3} <- KEY PARAMETER")
    else:
        print(f"{key:>20}: {value:>3}")

print("\nCOMMON ADJUSTMENTS:")
print("1. NOISY DATA: increase threshold_std_prim to 6-8")
print("2. WEAK DIFFRACTION: decrease threshold_std_prim to 3-4")
print("3. STRICT QUALITY: increase good_rule to 4-5")
print("4. LENIENT QUALITY: decrease good_rule to 2")

# Initialize the analyzer
analyzer = DiffractionAnalyzer(params)
print("\nAnalyzer initialized with default parameters")
print("Ready to process MRC files")

In [ ]:
analyzer = DiffractionAnalyzer(params)

# Process folders
current_path = os.getcwd()
for folder in os.listdir(current_path):
    if os.path.isdir(folder) and (folder == 'snapshots' or '_' in folder):
        print(f"\nProcessing folder: {folder}")
        results = analyzer.process_directory(folder, visualize=True)